# Sycomp Onboarding - Session 1

0. Platform Architecture + K8s Background

1. Iguazio Overview

2. Code to Microservice

3. More on MLRun Functions

## 0. Platform Architecture + Docker/K8s Background

![Docker](img/docker.png)
- Docker
    - A container is a standard unit of software that packages up code and all its dependencies so the application runs quickly and reliably from one computing environment to another.
    - A Docker container image is a lightweight, standalone, executable package of software that includes everything needed to run an application: code, runtime, system tools, system libraries and settings.

![K8s](img/k8s.png)
- Kubernetes (also known as K8s)
    - Open-source system for automating deployment, scaling, and management of containerized applications.
    - One stop shop - manages compute, networking, storage, permissions, scaling, etc.

- Platform Architecture
![Platform Architecture](img/platform_architecture.png)

## 1. Iguazio Overview
- UI Tour
- Project Management
- Managed Services

## 2. Code to Microservice
- Job Example (Python)
- Job Example (Notebook)
- Nuclio Example

- Package Python code into microservice to run on K8s

- ```python
fn = code_to_function(
    name="test",
    project="my_project",
    filename="test.py", # .py or .ipynb
    kind="job",
    image="mlrun/mlrun",
    handler="my_func"
)
fn.run()
```

- Add hooks for experiment tracking and more

- Experiment tracking
    ```python
    context.log_model()
    context.log_dataset()
    context.log_result()
    ```

- Pass inputs/parameters
    ```python
    context.get_param("batch_size")
    context.get_input("training_set")
    ```

- Logging
    ```python
    context.logger.info("Something happened")
    context.logger.error("Something went wrong")
    ```

### Job Example - Python

In [1]:
from mlrun import code_to_function

job = code_to_function(
    name="my_job",
    project="kitchen-sink-demo",
    filename="assets/job.py",
    kind="job",
    image="mlrun/mlrun",
    handler="my_func"
)
job.run(params={"value" : 123}, local=True)

> 2021-09-08 22:19:35,359 [info] starting run my-job-my_func uid=4ae25640aad44007a9ff8c2b57b56196 DB=http://mlrun-api:8080
> 2021-09-08 22:19:35,500 [info] Logging value
> 2021-09-08 22:19:35,512 [info] Logging CSV


Converting input from bool to <class 'numpy.uint8'> for compatibility.


> 2021-09-08 22:19:35,831 [info] Logging model


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
kitchen-sink-demo,...57b56196,0,Sep 08 22:19:35,completed,my-job-my_func,v3io_user=nickkind=owner=nickhost=jupyter-nick-55654b7c59-q5zvd,,value=123,value=123,my_csvmy_model


to track results use .show() or .logs() or in CLI: 
!mlrun get run 4ae25640aad44007a9ff8c2b57b56196 --project kitchen-sink-demo , !mlrun logs 4ae25640aad44007a9ff8c2b57b56196 --project kitchen-sink-demo
> 2021-09-08 22:19:36,022 [info] run executed, status=completed


### Job Example - Notebook

In [2]:
from mlrun import auto_mount

job = code_to_function(
    name="my_notebook",
    project="kitchen-sink-demo",
    filename="assets/notebook.ipynb",
    kind="job",
    image="mlrun/mlrun",
    handler="main"
)
job.apply(auto_mount())
job.run(params={"test_size" : 0.3, "gamma" : 0.001, "C" : 100}, local=False)

> 2021-09-08 22:20:03,185 [info] starting run my-notebook-main uid=d3c3801cd97043d9bfe92009eac8285c DB=http://mlrun-api:8080
> 2021-09-08 22:20:03,351 [info] Job is running in the background, pod: my-notebook-main-wpjw4
> 2021-09-08 22:20:08,163 [info] Getting data
> 2021-09-08 22:20:08,168 [info] Training model
> 2021-09-08 22:20:08,170 [info] Evaluating model
> 2021-09-08 22:20:08,171 [info] Logging datasets
> 2021-09-08 22:20:08,480 [info] Logging model
> 2021-09-08 22:20:08,598 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
kitchen-sink-demo,...eac8285c,0,Sep 08 22:20:08,completed,my-notebook-main,v3io_user=nickkind=jobowner=nickhost=my-notebook-main-wpjw4,,test_size=0.3gamma=0.001C=100,accuracy=0.9555555555555556,X_trainX_testy_trainy_testnotebook_model


to track results use .show() or .logs() or in CLI: 
!mlrun get run d3c3801cd97043d9bfe92009eac8285c --project kitchen-sink-demo , !mlrun logs d3c3801cd97043d9bfe92009eac8285c --project kitchen-sink-demo
> 2021-09-08 22:20:09,503 [info] run executed, status=completed


### Real-Time (Nuclio) Example

In [3]:
rt = code_to_function(
    name="my_rt",
    project="kitchen-sink-demo",
    filename="assets/rt.py",
    kind="nuclio",
    image="mlrun/mlrun",
    handler="handler"
)
rt.deploy()

> 2021-09-08 22:20:14,407 [info] Starting remote function deploy
2021-09-08 22:20:14  (info) Deploying function
2021-09-08 22:20:14  (info) Building
2021-09-08 22:20:14  (info) Staging files and preparing base images
2021-09-08 22:20:14  (info) Building processor image
2021-09-08 22:20:15  (info) Build complete
2021-09-08 22:20:17  (info) Function deploy complete
> 2021-09-08 22:20:18,100 [info] function deployed, address=default-tenant.app.us-sales.iguazio-cd0.com:32249


'http://default-tenant.app.us-sales.iguazio-cd0.com:32249'

In [4]:
%%time
rt.invoke(path="/")

CPU times: user 8.68 ms, sys: 0 ns, total: 8.68 ms
Wall time: 20.8 ms


{'result': '0.60113840011245'}

## 3. More on MLRun Functions
- Basic configuration - Limits/requests
- Import/export
- Marketplace integration

### Basic Configuration - Limits/Requests

https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/#requests-and-limits

In [5]:
from mlrun import auto_mount

job = code_to_function(
    name="my_notebook",
    project="kitchen-sink-demo",
    filename="assets/notebook.ipynb",
    kind="job",
    image="mlrun/mlrun",
    handler="main"
)
job.apply(auto_mount())
job.with_requests(cpu=2)
job.with_limits(cpu=2)
job.run(params={"test_size" : 0.3, "gamma" : 0.001, "C" : 100}, local=False)

> 2021-09-08 22:20:45,510 [info] starting run my-notebook-main uid=1961378eb63e4896b0cdcf699ca1febb DB=http://mlrun-api:8080
> 2021-09-08 22:20:45,820 [info] Job is running in the background, pod: my-notebook-main-vg4hg
> 2021-09-08 22:20:50,924 [info] Getting data
> 2021-09-08 22:20:50,929 [info] Training model
> 2021-09-08 22:20:50,930 [info] Evaluating model
> 2021-09-08 22:20:50,932 [info] Logging datasets
> 2021-09-08 22:20:51,223 [info] Logging model
> 2021-09-08 22:20:51,354 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
kitchen-sink-demo,...9ca1febb,0,Sep 08 22:20:50,completed,my-notebook-main,v3io_user=nickkind=jobowner=nickhost=my-notebook-main-vg4hg,,test_size=0.3gamma=0.001C=100,accuracy=0.9777777777777777,X_trainX_testy_trainy_testnotebook_model


to track results use .show() or .logs() or in CLI: 
!mlrun get run 1961378eb63e4896b0cdcf699ca1febb --project kitchen-sink-demo , !mlrun logs 1961378eb63e4896b0cdcf699ca1febb --project kitchen-sink-demo
> 2021-09-08 22:20:52,019 [info] run executed, status=completed


### Export

In [6]:
# print(rt.to_yaml())

In [7]:
rt.export()

> 2021-09-08 22:21:09,774 [info] function spec saved to path: function.yaml


### Import (File)

In [8]:
from mlrun import import_function

In [9]:
new_rt = import_function("function.yaml")

In [10]:
# print(new_rt.to_yaml())

### Import (Marketplace)

In [11]:
# Reaches out to https://github.com/mlrun/functions (customizable)
fn = import_function("hub://describe")

In [12]:
# print(fn.to_yaml())